In [2]:
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

filepath = 'data/ss2.png'
image = Image.open(filepath).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRC

In [3]:
generated_ids = model.generate(pixel_values, max_new_tokens=1)

In [4]:
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

'Test'

In [2]:
!pip install quanto

  Obtaining dependency information for quanto from https://files.pythonhosted.org/packages/06/22/89e363878024a21b0093f69c7040c33715ff52cd0cbfb1ac91a0977f511f/quanto-0.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig

quantization_config = QuantoConfig(load_in_8bit=True)


# Load the LLaMA model and tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=quantization_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
llama_model.device

device(type='mps', index=0)

In [12]:
# Feed the current token to LLaMA
prompt = f"Generate the next token after: {generated_text}"
input_ids = llama_tokenizer(prompt, return_tensors="pt").input_ids.to(llama_model.device)
input_ids

tensor([[128000,  32215,    279,   1828,   4037,   1306,     25,   3475]],
       device='mps:0')

In [18]:
# Generate the next token
llama_output = llama_model.generate(input_ids, max_new_tokens=1)
predicted_token = llama_tokenizer.decode(llama_output[0][-1], skip_special_tokens=True)
print("Predicted token from LLaMA:", predicted_token)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted token from LLaMA:  the


In [50]:
# Add the token predicted by LLaMA to TrOCR's decoder input
decoder_input_ids = processor.tokenizer(predicted_token, return_tensors="pt").input_ids
decoder_input_ids

tensor([[   0, 4500,    5,    2]])

In [53]:
generated_ids = model.generate(
    pixel_values,
    decoder_input_ids=decoder_input_ids,
    max_new_tokens=1
)
generated_ids

tensor([[   2,    0, 4500,    5,    2, 7440]])

In [52]:
next_token = processor.batch_decode(generated_ids, skip_special_tokens=True)[-1]
print("Next token predicted by TrOCR:", next_token)

Next token predicted by TrOCR:  Test the valuation chats ( once I


In [39]:
import numpy as np

vocab_size = 50265

all_ids = np.arange(vocab_size).reshape(1, -1)

# Get the corresponding text for each token ID
roberta_tokens = []
for id in all_ids[0]:
    roberta_tokens.append(processor.tokenizer.decode(id))
    
roberta_tokens[300:340]

[' got',
 ' life',
 ' Monday',
 ' found',
 ' use',
 ' W',
 '4',
 ' Wednesday',
 ' own',
 ' according',
 ' play',
 ' show',
 ' St',
 ' man',
 ' left',
 ' United',
 ' 12',
 ' place',
 ' If',
 ' lot',
 ' former',
 ' 0',
 ').',
 ' support',
 'ie',
 ' billion',
 ' t',
 ' shares',
 '!',
 'z',
 'k',
 ' State',
 ' points',
 ' group',
 ' school',
 ' information',
 ' 2016',
 'al',
 'r',
 ' win']

In [59]:
# Get text for all llama tokens

llama_vocab_size = llama_tokenizer.vocab_size

llama_all_ids = np.arange(llama_vocab_size).reshape(1, -1)

llama_tokens = []
for id in llama_all_ids[0]:
    llama_tokens.append(llama_tokenizer.decode(id))

llama_tokens[3475]

' Test'

In [66]:
llama_to_roberta = [[] for _ in range(llama_vocab_size)]

for llama_id in range(llama_vocab_size):
    if llama_id % 10000 == 0:
        print(f"Processing token {llama_id}...")
    llama_token = llama_tokens[llama_id]
    for roberta_id, roberta_token in enumerate(roberta_tokens):
        if roberta_token.startswith(llama_token) or llama_token.startswith(roberta_token):
            llama_to_roberta[llama_id].append(roberta_id)
            
llama_to_roberta[100:200]

Processing token 0...
Processing token 10000...
Processing token 20000...
Processing token 30000...
Processing token 40000...
Processing token 50000...
Processing token 60000...
Processing token 70000...
Processing token 80000...
Processing token 90000...
Processing token 100000...
Processing token 110000...
Processing token 120000...


[[17,
  27,
  46,
  48,
  649,
  711,
  862,
  1277,
  1376,
  2023,
  2469,
  2840,
  3070,
  3602,
  3726,
  3849,
  4056,
  4333,
  4394,
  4726,
  4958,
  5543,
  5782,
  6163,
  6248,
  6382,
  6569,
  6800,
  7258,
  7471,
  7487,
  8188,
  8210,
  8384,
  8906,
  9085,
  9253,
  9264,
  9357,
  9470,
  10172,
  10278,
  10470,
  10659,
  10674,
  10809,
  10965,
  11423,
  11582,
  11936,
  12410,
  12736,
  12798,
  13859,
  14238,
  14285,
  14292,
  15113,
  15264,
  15375,
  15389,
  15722,
  16948,
  17772,
  18164,
  18400,
  18537,
  18636,
  19002,
  20024,
  21402,
  21593,
  21955,
  22063,
  22919,
  23133,
  23171,
  24107,
  24987,
  25417,
  25448,
  26068,
  27819,
  27969,
  30151,
  31193,
  35423,
  35861,
  36484,
  36714,
  37127,
  38155,
  38718,
  39817,
  40669,
  40756,
  41335,
  41735,
  41907,
  42088,
  42348,
  42358,
  42393,
  43251,
  43621,
  43998,
  44025,
  44148,
  44636,
  45209,
  45262,
  45449,
  45704,
  45895,
  46015,
  46030,
  46164

In [72]:
# Save the mapping
import json

with open("llama_to_roberta.json", "w") as f:
    json.dump(llama_to_roberta, f)
    